In [29]:
import json;
import pandas as pd;
import numpy as np;

In [127]:
paths = {}
colors = ['Yellow','Blue','Red','Green','Silver','Orange']
for c in colors:
    paths[c] = pd.DataFrame(eval(json.load(open(c.lower()+".txt"))))
    paths[c] = paths[c].sort_values(['SeqNum'])

In [129]:
def getLineColors(station):
    colors = [];
    for n in paths:
        thisLine = paths[n];
        if(len(thisLine[thisLine["StationName"]==station]['LineCode'])>0):
            colors.append(n)
    return colors;

In [130]:
getLineColors('Metro Center')

['Orange', 'Red', 'Blue', 'Silver']

In [131]:
origin = 'U Street\\/African-Amer Civil War Memorial\\/Cardozo'
destination = 'Braddock Road'

In [132]:
a=getLineColors(origin)

In [133]:
b=getLineColors(destination)

In [191]:
def getPossiblePaths(start,end):
    path_possibilities = {};
#     direct path, if it exists
    sharedLines = list(set(getLineColors(start)).intersection(getLineColors(end)));
    if(len(sharedLines)>0):
        for n in sharedLines:
            origin_index = int(list(paths[n][paths[n]['StationName']==start].index)[0])
            destination_index = int(list(paths[n][paths[n]['StationName']==end].index)[0])
            if(origin_index>destination_index):
                the_path = list(paths[n].iloc[destination_index:origin_index+1]['StationName']);
                the_path.reverse();
                path_possibilities[n]=(the_path);
            else:
                path_possibilities[n] = (list(paths[n].iloc[origin_index:destination_index+1]['StationName']));
                
    return path_possibilities;

In [221]:
def findTransferPoints(start,destination):
    possible_transfer_points = {};
    start_lines = getLineColors(start);
    dest_lines = getLineColors(destination);
    for p in start_lines:
        possible_transfer_points[p]={};
        for q in dest_lines:
            intersections = set(list(paths[p]['StationName'])).intersection(list(set(paths[q]['StationName'])))
            if(p!=q):
                possible_transfer_points[p][q] = (intersections);
    return possible_transfer_points;

In [223]:
tfs = findTransferPoints(origin,destination)

In [228]:
tfs['Green']

{'Blue': {"L'Enfant Plaza"},
 'Yellow': {'Archives-Navy Memorial-Penn Quarter',
  'Columbia Heights',
  'Fort Totten',
  'Gallery Pl-Chinatown',
  'Georgia Ave-Petworth',
  "L'Enfant Plaza",
  'Mt Vernon Sq 7th St-Convention Center',
  'Shaw-Howard U',
  'U Street\\/African-Amer Civil War Memorial\\/Cardozo'}}

In [230]:
findTransferPoints('Metro Center','Waterfront')

{'Blue': {'Green': {"L'Enfant Plaza"}},
 'Orange': {'Green': {"L'Enfant Plaza"}},
 'Red': {'Green': {'Fort Totten', 'Gallery Pl-Chinatown'}},
 'Silver': {'Green': {"L'Enfant Plaza"}}}

In [232]:
findTransferPoints('Union Station','Potomac Ave')
# This should show Gallery Pl -> L'Enfant Plaza

{'Red': {'Blue': {'Metro Center'},
  'Orange': {'Metro Center'},
  'Silver': {'Metro Center'}}}

In [387]:
def findDoubleTransferPoints(start,destination):
    temp_result = [];
    possible_transfer_points = {};
    startLineStations = list(set([k for k in paths[n]['StationName'] for n in getLineColors(start)]))
    destLineStations = list(set([g for g in paths[q]['StationName'] for q in getLineColors(destination)]))
#     return stations on origin_line that share a line color with any of the stations on dest_line
    startLineStationColors = {station:getLineColors(station) for station in startLineStations}
    destLineStationColors = {station:getLineColors(station) for station in destLineStations}
    return (startLineStationColors,destLineStationColors)

            

In [389]:
starts = findDoubleTransferPoints("Union Station","Potomac Ave")[0]
dests = findDoubleTransferPoints("Union Station","Potomac Ave")[1]

In [417]:
flat = []
for k in list(starts.values()):
    for g in k:
        flat.append(g)
set(flat)

{'Blue', 'Green', 'Orange', 'Red', 'Silver', 'Yellow'}

In [421]:
set(paths['Red']['StationName']).intersection(set(paths['Blue']['StationName']))

{'Metro Center'}

In [438]:
transfer_points = {};
for n in colors:
    transfer_points[n]={}
    for k in colors:
        if(k!=n):
            transfer_points[n][k]=list(set(paths[n]['StationName']).intersection(set(paths[k]['StationName'])))

In [426]:
transfer_points

{'Blue': {'Green': ["L'Enfant Plaza"],
  'Orange': ['Potomac Ave',
   "L'Enfant Plaza",
   'McPherson Square',
   'Farragut West',
   'Capitol South',
   'Smithsonian',
   'Foggy Bottom-GWU',
   'Rosslyn',
   'Eastern Market',
   'Stadium-Armory',
   'Federal Center SW',
   'Federal Triangle',
   'Metro Center'],
  'Red': ['Metro Center'],
  'Silver': ['Potomac Ave',
   "L'Enfant Plaza",
   'McPherson Square',
   'Morgan Boulevard',
   'Farragut West',
   'Capitol South',
   'Smithsonian',
   'Addison Road-Seat Pleasant',
   'Foggy Bottom-GWU',
   'Rosslyn',
   'Benning Road',
   'Eastern Market',
   'Stadium-Armory',
   'Federal Center SW',
   'Capitol Heights',
   'Largo Town Center',
   'Federal Triangle',
   'Metro Center'],
  'Yellow': ['Braddock Road',
   "L'Enfant Plaza",
   'Pentagon City',
   'Crystal City',
   'King St-Old Town',
   'Ronald Reagan Washington National Airport',
   'Pentagon']},
 'Green': {'Blue': ["L'Enfant Plaza"],
  'Orange': ["L'Enfant Plaza"],
  'Red': ['F

In [437]:
for n in getLineColors(start):
    for k in getLineColors(destination):
        if(n!=k):
            print(" ")
            print(transfer_points[n][k])

 
["L'Enfant Plaza"]
 
["L'Enfant Plaza"]
 
["L'Enfant Plaza"]


In [448]:
for n in getLineColors("Union Station"):
    for k in getLineColors("Potomac Ave"):
        if(n!=k):
            print(" ")
            print(transfer_points[n][k]) 

 
['Metro Center']
 
['Metro Center']
 
['Metro Center']


In [465]:
graph = {};
paths['Yellow']['StationName']

for n in colors:
    for p in paths[n]['StationName']:
        graph[p]=[]
graph

{'Addison Road-Seat Pleasant': [],
 'Anacostia': [],
 'Archives-Navy Memorial-Penn Quarter': [],
 'Arlington Cemetery': [],
 'Ballston-MU': [],
 'Benning Road': [],
 'Bethesda': [],
 'Braddock Road': [],
 'Branch Ave': [],
 'Brookland-CUA': [],
 'Capitol Heights': [],
 'Capitol South': [],
 'Cheverly': [],
 'Clarendon': [],
 'Cleveland Park': [],
 'College Park-U of MD': [],
 'Columbia Heights': [],
 'Congress Heights': [],
 'Court House': [],
 'Crystal City': [],
 'Deanwood': [],
 'Dunn Loring-Merrifield': [],
 'Dupont Circle': [],
 'East Falls Church': [],
 'Eastern Market': [],
 'Eisenhower Avenue': [],
 'Farragut North': [],
 'Farragut West': [],
 'Federal Center SW': [],
 'Federal Triangle': [],
 'Foggy Bottom-GWU': [],
 'Forest Glen': [],
 'Fort Totten': [],
 'Franconia-Springfield': [],
 'Friendship Heights': [],
 'Gallery Pl-Chinatown': [],
 'Georgia Ave-Petworth': [],
 'Glenmont': [],
 'Greenbelt': [],
 'Greensboro': [],
 'Grosvenor-Strathmore': [],
 'Huntington': [],
 'Judici

In [473]:
for z in graph.keys():
    for j in getLineColors(z):
        listOfStations = list(paths[j]['StationName'])
        currentIndex = listOfStations.index(z);
#         print (currentIndex)
        if(currentIndex>0):
#             set prev
            graph[z].append(listOfStations[currentIndex-1]);
        if(currentIndex<len(listOfStations)-1):
            graph[z].append(listOfStations[currentIndex+1])
    graph[z]=list(set(graph[z]))

In [474]:
graph

{'Addison Road-Seat Pleasant': ['Morgan Boulevard', 'Capitol Heights'],
 'Anacostia': ['Congress Heights', 'Navy Yard-Ballpark'],
 'Archives-Navy Memorial-Penn Quarter': ["L'Enfant Plaza",
  'Gallery Pl-Chinatown'],
 'Arlington Cemetery': ['Pentagon', 'Rosslyn'],
 'Ballston-MU': ['East Falls Church', 'Virginia Square-GMU'],
 'Benning Road': ['Stadium-Armory', 'Capitol Heights'],
 'Bethesda': ['Friendship Heights', 'Medical Center'],
 'Braddock Road': ['King St-Old Town',
  'Ronald Reagan Washington National Airport'],
 'Branch Ave': ['Suitland'],
 'Brookland-CUA': ['Rhode Island Ave-Brentwood', 'Fort Totten'],
 'Capitol Heights': ['Benning Road', 'Addison Road-Seat Pleasant'],
 'Capitol South': ['Eastern Market', 'Federal Center SW'],
 'Cheverly': ['Deanwood', 'Landover'],
 'Clarendon': ['Court House', 'Virginia Square-GMU'],
 'Cleveland Park': ['Van Ness-UDC', 'Woodley Park-Zoo\\/Adams Morgan'],
 'College Park-U of MD': ["Prince George's Plaza", 'Greenbelt'],
 'Columbia Heights': ['Ge

In [ ]:

# https://www.python.org/doc/essays/graphs/
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return [path]
        if not graph.has_key(start):
            return []
        paths = []
        for node in graph[start]:
            if node not in path:
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths